In [ ]:
# this file is used to generated the sparse part data
# because we are working in product state basis, this will be huge, we HAVE to work in the sparse matrix format

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from jax.experimental import sparse
jax.config.update("jax_enable_x64", True)
from scipy.sparse.linalg import eigsh
from scipy import sparse
from numpy import genfromtxt
# initialize variables
N = 2 # number of sites
half_filling = True

if half_filling == True:

    N_e = N # total number of electrons
    S_z = 0
W = 2*N # each sites can have 2 electrons
# Q is the dimension
Q = pow(4, N)

# decimal number to binary array function
def D2B(num):
    string = f'{num:1b}'
    result = np.zeros(W-len(string), int)

    for ele in string:
        result = np.append(result, int(ele))

    return result #nd array


# binary array to decimal function
def B2D(array):
    res = 0
    for ele in array:
        res = (res << 1) | ele
    return res[0]

In [ ]:
# # initialize t matrix
# t_matrix = np.zeros((N, N), dtype='float64')

# global t_value
# t_value = -1
# global t_value_nn
# t_value_nn = -0  # hop between 1 and 3 etc



# # set PBC case
# PBC_condition = True

# for i in range(N):
#     if PBC_condition == True:
#         t_matrix[0][-1] = t_value
#     for j in range(N):
#         if j == (i + 1) : # hopping matrix nn
#             #t_matrix[i][j] = t_value+np.random.uniform(0,0.4,1)
#             t_matrix[i][j] = t_value
#         if j == (i + 2) : # hopping matrix nnn
#             #t_matrix[i][j] = t_value+np.random.uniform(0,0.4,1)
#             t_matrix[i][j] = t_value_nn
# # add another half back
# t_matrix = np.transpose(t_matrix) + t_matrix
# t_matrix

In [ ]:
# create dict for new index in N_e sector
old_index_list=[]
for i in range(Q):
    state = D2B(i)
    up_total=0
    down_total=0
    state_4_U = state.reshape((N, 2)) # state reshape to an array with 2 numbers for each site
    #print(state_4_U)
    for j in range(N):
        up_total = up_total + state_4_U[j][0] # sum up all the first element
        down_total = down_total + state_4_U[j][1] # sum up all the second element
    if (np.sum(state) == N_e) and (up_total-down_total==S_z): # half filling condition
        old_index_list.append(i)


old_index_list = np.array(old_index_list)
print('dimension is',len(old_index_list))

dimension is 4


In [ ]:
# save this data
if N == 9:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/old_index_list={N}_Ne_{N_e}.txt'
else:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/old_index_list={N}.txt'
np.savetxt(file_path, old_index_list, fmt='%d')

In [ ]:
# The U part does not need matrix to do calculation it just attach a phase to the state vector later

In [ ]:
unit_U_matrix_row_list=[] # diagonal matrix anyways hence row is col as well, dimension is len( old_index_list )
double_occ_site_index_list=[]
for index, value in enumerate(old_index_list):

    state = D2B(value)
    # U part
    # reshape it so that each array for one electron
    state_4_U = state.reshape((N, 2))
    # loop each array to see if how many double occuplied
    number_of_double_occ_list = []
    for j in range(N):
        one_electron_array = state_4_U[j]
        one_electron_array_sum = np.sum(one_electron_array)
        if one_electron_array_sum ==2:
            number_of_double_occ_list.append(j)
    if len(number_of_double_occ_list)!=0:
        double_occ_site_index_list.append(number_of_double_occ_list)
        unit_U_matrix_row_list.append(index)

In [ ]:
# explain : unit_U_matrix_row_list is the list that gives the index of elements in old_index_list
# that does not have double occ
# For example, in N=4, it does not have 11 or 14,
# bc D2B(old_index_list[11]) gives [0, 1, 0, 1, 1, 0, 1, 0]
# D2B(old_index_list[14]) gives  [0, 1, 1, 0, 1, 0, 0, 1]
# double_occ_site_index_list is the site index for that state, which site has double occ

In [ ]:
def get_sprase_U_matrix(input_array): #input array is the U value for each site
    row = unit_U_matrix_row_list
    col = unit_U_matrix_row_list

    # get U value, number of double occ times the corresponding U value
    data = np.zeros(len(row))
    for index in range(len(double_occ_site_index_list)):
        value = double_occ_site_index_list[index]
        total_U_for_one_state = 0
        for site_index in value:
            total_U_for_one_state = total_U_for_one_state + input_array[site_index]
        data[index] = total_U_for_one_state

    sprase_U_matrix = sparse.coo_matrix((data, (row, col)), shape=(len(old_index_list), len(old_index_list) ) ) # len of old is full dimension
    # row col is the ones with non zero U value

    return sprase_U_matrix


In [ ]:
new_index_list = np.arange(0, len(old_index_list))
index_dict = dict(zip(old_index_list, new_index_list))

# Initialize the dictionary for hopping pairs
hopping_dict = {}
for i in range(N):
    for j in range(i + 1, N):
        # Include keys for both positive and negative spins
        hopping_dict[(i, j, 1)] = {'row': [], 'col': [], 'data': []}  # Key for positive spin
        hopping_dict[(i, j, -1)] = {'row': [], 'col': [], 'data': []}  # Key for negative spin
# Start looping each state and see what other states it couples to
for index, value in enumerate(old_index_list):

    state = D2B(value)
    # Reshape it so that each array is for one electron
    state_4_U = state.reshape((N, 2))

    # t part

    # Where it starts to hop
    for j in range(N):
        # Loop where it hops to
        for k in range(N):
            if j < k:
                # Up spin part
                spin_sign = 1
                if state_4_U[j][0] == 1 and state_4_U[k][0] == 0:
                    number_pass = 0
                    hop_state = np.copy(state_4_U)  # A copy of the state so that it won't change the original one
                    state_to_count = np.copy(state)
                    start_position = 2 * j
                    # Count how many from the edge to start position
                    number_pass += np.sum(state_to_count[:start_position])
                    # Remove that one
                    state_to_count[start_position] = 0
                    # Count how many from the edge to end position
                    end_position = 2 * k
                    number_pass += np.sum(state_to_count[:end_position])
                    hop_state[j][0] = 0
                    hop_state[k][0] = 1
                    # New number
                    hop_state = hop_state.reshape((2 * N, 1))
                    new_num = B2D(hop_state)
                    # For hopping matrix
                    hopping_dict[(j, k,spin_sign)]['row'].append(index)
                    hopping_dict[(j, k,spin_sign)]['col'].append(index_dict[new_num])
                    hopping_dict[(j, k,spin_sign)]['data'].append(pow(-1, number_pass))

                # Down spin part
                spin_sign = -1
                if state_4_U[j][1] == 1 and state_4_U[k][1] == 0:
                    number_pass = 0
                    hop_state = np.copy(state_4_U)  # A copy of the real state so that it won't change the original one
                    state_to_count = np.copy(state)
                    start_position = 2 * j + 1
                    # Count how many from the edge to start position
                    number_pass += np.sum(state_to_count[:start_position])
                    # Remove that one
                    state_to_count[start_position] = 0
                    # Count how many from the edge to end position
                    end_position = 2 * k + 1
                    number_pass += np.sum(state_to_count[:end_position])
                    hop_state[j][1] = 0
                    hop_state[k][1] = 1
                    # New number
                    hop_state = hop_state.reshape((2 * N, 1))
                    new_num = B2D(hop_state)
                    # For hopping matrix
                    hopping_dict[(j, k,spin_sign)]['row'].append(index)
                    hopping_dict[(j, k,spin_sign)]['col'].append(index_dict[new_num])
                    hopping_dict[(j, k,spin_sign)]['data'].append(pow(-1, number_pass))

#numpy method
#Convert lists to sparse matrices in the dictionary
for key in hopping_dict:
    row = np.array(hopping_dict[key]['row'])
    col = np.array(hopping_dict[key]['col'])
    data = np.array(hopping_dict[key]['data'])
    hopping_dict[key] = sparse.coo_matrix((data, (row, col)), shape=(len(old_index_list), len(old_index_list)))

# #hopping_dict now contains sparse matrices for each hopping pair


In [ ]:
# save the hopping dict
import pickle
# Specify the file path to save the dictionary
if N == 9:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/torch_sparse_matrices_dict_N={N}_Ne_{N_e}.pkl'
else:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/torch_sparse_matrices_dict_N={N}.pkl'

with open(file_path, 'wb') as f:
    pickle.dump(hopping_dict, f)

In [ ]:
# # Get indices of non-zero elements
# non_zero_indices = np.nonzero(t_matrix)

# # Create a set to store unique pairs
# unique_indices_set = set()

# # Add pairs to the set, ensuring each pair is unique
# for i, j in zip(non_zero_indices[0], non_zero_indices[1]):
#     if i < j:
#         unique_indices_set.add((i, j))
#     elif i > j:
#         unique_indices_set.add((j, i))

# # Convert set back to list
# t_matrix_hopping_pair = list(unique_indices_set)


In [ ]:
# t_matrix_hopping_pair

In [ ]:
# # Get sparse matrix serve as check
# # CAREFUL, ONLY DOING NN HOPPING HERE SO JUST t_value
# sparse_hopping_matrix = t_value * (
#     hopping_dict[(0, 1, 1)] + hopping_dict[(0, 1, 1)].T +
#     hopping_dict[(0, 1, -1)] + hopping_dict[(0, 1, -1)].T
# )

# # Loop through t_matrix_hopping_pair and add matching pairs from hopping_dict
# for pair in t_matrix_hopping_pair:
#     for key in hopping_dict:
#         if pair == key[:2] and pair != (0,1):# Check if the first two elements match
#             #print(key)
#             sparse_hopping_matrix = sparse_hopping_matrix + t_value * (hopping_dict[key] + hopping_dict[key].T)

# # sparse_hopping_matrix now contains the sum of all matching pairs

In [ ]:
# #full h
# U_value = 8
# h_sprase = sparse_hopping_matrix + get_sprase_U_matrix(np.ones(N)*U_value)
# # h_sprase
# # print(sparse.linalg.eigsh(h_sprase, k=1, which='SA'))

In [ ]:
# sparse.save_npz(f'/content/drive/My Drive/Hubbard_2024/product_state/torch_h_sparse_N_{N}_U_{U_value}.npz', h_sprase)

In [ ]:
# ## test
# import jax.numpy as jnp
# from jax.scipy.sparse import coo_matrix

In [ ]:
# the U part code

# get the potential list for each number operator in product state basis bascailly to check if there's one or 2 eletrons
number_op_list = []
for i in range(N): # this is for U term, loop N which is each site
    number_occ_list =[]

    for index, value in enumerate(old_index_list):
        state = D2B(value)
        # reshape it so that each array for one electron
        state_4_U = state.reshape((N, 2))
        #print(state_4_U)

        # i is the site index

        number_occ = np.sum(state_4_U[i])

        if number_occ ==2:
            number_occ_list.append(1) # number of doulbe occ
        else:
            number_occ_list.append(0)

    number_op_list.append(number_occ_list)


# loop each site, bc the parameter is at each site
for i in range(N): # this is for ci dagger ci

    number_occ_list =[]
    for index, value in enumerate(old_index_list):
        state = D2B(value)
        # reshape it so that each array for one electron
        state_4_U = state.reshape((N, 2))
        #print(state_4_U)

        # i is the site index

        number_occ = np.sum(state_4_U[i])
        number_occ_list.append(number_occ)

    number_op_list.append(number_occ_list)

In [ ]:
number_op_list = np.array(number_op_list)
if N == 9:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/number_op_list_N={N}_Ne_{N_e}.txt'
else:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/number_op_list_N={N}.txt'

np.savetxt(file_path, number_op_list,fmt='%d')

In [ ]:
# Initialize the dictionary for hopping pairs
hopping_cos_dict = {}
for i in range(N):
    for j in range(i + 1, N):
        # Include keys for both positive and negative spins
        hopping_cos_dict[(i, j, 1)] = []  # Key for positive spin
        hopping_cos_dict[(i, j, -1)] = []  # Key for negative spin

# Populate the dictionary
for (i, j, spin) in hopping_cos_dict.keys():
    good_index_list = []
    for index, value in enumerate(old_index_list):
        state = D2B(value)
        state_4_U = state.reshape((N, 2))

        if spin == 1:
            spin_list = state_4_U.T[0]  # Spin-up list
        else:
            spin_list = state_4_U.T[1]  # Spin-down list

        if np.sum(spin_list[[i, j]]) == 1:  # Sum is 1 means it can hop
            good_index_list.append(index)

    # Update the dictionary
    hopping_cos_dict[(i, j, spin)] = np.array(good_index_list)

# Now hopping_cos_dict contains the good index lists for all (i, j, spin) pairs

In [ ]:
# Specify the file path to save the dictionary
if N == 9:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/torch_hopping_cose_dict_N={N}_Ne_{N_e}.pkl'
else:
    file_path = f'/content/drive/My Drive/Hubbard_2024/2d/torch_hopping_cose_dict_N={N}.pkl'

with open(file_path, 'wb') as f:
    pickle.dump(hopping_cos_dict, f)